# 基于 MindQuantum 0.7.0 实现半导体双量子点下，采用多基矢编码方法 MBE 实现最大割求解

## 2-qubit QD MBE-VQE

In [ ]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

def _matrix_(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_(coeff):
    return -1j*_matrix_(coeff)@(s_z*dt)

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*ddt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*ddt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10

cz_params = np.array( [1.5472503,  1.4179231,  1.540713,   1.9724044,  1.9253408,  1.3879265,
                     0.8130467,  0.76446086, 1.2703444,  1.8553745,  1.0291328,  1.2492974,
                     0.7880994,  0.3026381,  0.31203356, 0.30834132, 0.9533752,  1.3802187,
                     1.270656,   0.5646567,  0.94619316, 0.97377133, 1.9658349,  0.83277696,
                     1.0190777,  0.90001523, 0.26008993, 0.16526282, 0.22249524, 1.1596956,
                     1.5285202,  0.4919534,  0.01645389, 0.02608137, 0.6504683,  0.31325826,
                     0.4486266,  0.8677286,  1.3571227,  1.4995408,  1.1248059,  0.5996333,
                     0.32797617, 0.54987127])
# params_name = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '0', '1', '2', '3', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119']

def cz_circ():
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[:10]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([1,0]), gate_c_0(cz_params[21]).on([1,0])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([1,0]), gate_c_1(cz_params[23]).on([1,0])])
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[24:34]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

def arb_circ(prefix='0'):
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate(f'arb0{i}').on(0) for i in range(12)])
    circ_ += Circuit([gate(0).on(1) for i in range(12)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate(0).on(0) for i in range(12)])
    circ_ += Circuit([gate(f'arb1{i}').on(1) for i in range(12)])
    circ_ += BarrierGate()
    circ_ = add_prefix(circ_, prefix)
    return circ_

def qlayer(prefix='0'):
    circ_ = Circuit()
    circ_ += arb_circ(prefix)
    circ_ += cz_circ()
    return circ_

class my_simulator: 
    def get_expectation_with_grad(self, hams, circ): # 输入为 hams, circ_right, simulator_left 等，这里就只用 hams 代替。
        sim = Simulator('projectq', 2) 
        
        def grad_ops(inputs): # 输入为各量子门的参数
            h = 1e-4
            grad = []
            sim.reset()
            sim.apply_circuit(circ, inputs)
            exceptation = [sim.get_expectation(ham).real for ham in hams]
            for i in range(inputs.size):
                params_p, params_n = copy.deepcopy(inputs), copy.deepcopy(inputs)
                params_p[i] += h
                params_n[i] -= h
                sim.reset()
                sim.apply_circuit(circ, params_p)
                g_p = [sim.get_expectation(ham) for ham in hams]
                sim.reset()
                sim.apply_circuit(circ, params_n)
                g_n = [sim.get_expectation(ham) for ham in hams]
                grad.append([(p.real - n.real)/(2*h) for p, n in zip(g_p, g_n)])
                sim.reset()
            return exceptation, np.array(grad).T # 运行结果为期望值和梯度
        return grad_ops # MindQuantum 中，这里返回的是一个封装了 hams, grad_ops 等的一个封装器，这里简化为单独的 grad_ops

class ansatz_only_ops(nn.Cell):
    def __init__(self, expectation_with_grad):
        super().__init__()
        self.expectation_with_grad = expectation_with_grad
        self.shape_ops = operations.Shape()
        self.g = None 

    def construct(self, arg):
        fval, g_ans = self.expectation_with_grad(arg.asnumpy())
        self.g = np.real(g_ans)
        return ms.Tensor(np.real(fval), dtype=ms.float32)

    def bprop(self, arg, out, dout): 
        dout = dout.asnumpy()
        grad = dout @ self.g
        return ms.Tensor(grad, dtype=ms.float32)

class ansatz_only_layer(nn.Cell):
    def __init__(self, expectation_with_grad, weight='ones'):
        super().__init__()
        self.evolution = ansatz_only_ops(expectation_with_grad)
        weight_size = len(circ.params_name)
        self.weight = Parameter(initializer(weight, weight_size, dtype=ms.float32), name='ansatz_weight')
        self.abs = ops.Abs()

    def construct(self):
        return self.evolution(self.abs(self.weight))

class MyLoss(LossBase):
    def __init__(self, reduction='mean'):
        super(MyLoss, self).__init__(reduction)
        self.tanh = ops.Tanh()

    def construct(self, logits):
        out = self.tanh(logits)
        out = out[0] * out[2] + out[1] * out[3] + out[0] * out[1] 
        return self.get_loss(out)

class MyWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn):
       super(MyWithLossCell, self).__init__(auto_prefix=False)
       self._backbone = backbone
       self._loss_fn = loss_fn

    def construct(self):
       out = self._backbone()
       return self._loss_fn(out)

    @property
    def backbone_network(self):
       return self._backbone


circ = Circuit()
for i in range(2):
    circ += qlayer(prefix=f'{i}')

hams = [Hamiltonian(QubitOperator('Z0')), Hamiltonian(QubitOperator('X0')), 
        Hamiltonian(QubitOperator('Z1')), Hamiltonian(QubitOperator('X1'))]

my_sim = my_simulator()
grad_ops = my_sim.get_expectation_with_grad(hams, circ)

qnet = ansatz_only_layer(grad_ops)
loss = MyLoss()
net_with_criterion = MyWithLossCell(qnet, loss)
opti = Adam(qnet.trainable_params(), learning_rate=0.1) 
net = TrainOneStepCell(net_with_criterion, opti)

round = ops.Round()
loss_list = []
res_list = []
for i in range(100):
    loss = net()
    loss_list.append(loss)
    print(f'\n当前训练次数为：{i}, 损失函数值为：{loss}')
    out = qnet()
    res = (1 - round(out[0]) * round(out[2])) / 2 + (1 - round(out[1]) * round(out[3])) / 2 + (1 - round(out[0]) * round(out[1])) / 2
    print('MBE算法计算的最大割数为：', res) #int(res + 0.5))
    res_list.append(res) #int(res + 0.5))
    
print('\nloss_list:\n', loss_list)
print('\nres_list:\n', res_list)
 

In [ ]:
loss_array = np.array([])
for i in loss_list:
    loss_array = np.append(loss_array, i.asnumpy())
    
res_array = np.array([])
for i in res_list:
    res_array = np.append(res_array, i.asnumpy())
    
print('loss_array\n', loss_array)
print('\nres_array\n', res_array)

0.56820911 -0.06772007 -0.15884987  0.21334288  0.24638289 -0.01383337 -0.64901042 -0.77732551 -0.9761011  -0.83940554 -0.78345203 -0.81188357 -0.77375078 -0.87277997 -0.88260961 -0.92767239 -1.0147531  -1.0166626 -1.07208765 -1.0819068  -1.03471255 -1.03653717 -1.08419538 -1.09106088 -1.06723678 -1.06724095 -1.06764245 -1.09123957 -1.10459733 -1.07034826 -1.08677888 -1.09873986 -1.09860384 -1.09767449 -1.08537626 -1.09996045 -1.10696483 -1.09967256 -1.10074282 -1.10184681 -1.1062355  -1.10451984 -1.10361683 -1.10871792 -1.10590696 -1.10561097 -1.10770726 -1.10933506 -1.10946894 -1.10612607 -1.11011922 -1.10973358 -1.1099844  -1.1090343 -1.11044478 -1.11054432 -1.10974407 -1.11115432 -1.11102438 -1.1106931 -1.11028945 -1.111848   -1.11140621 -1.11092186 -1.11115718 -1.11181331 -1.11158633 -1.11131036 -1.11175632 -1.11166263 -1.11168683 -1.11176801 -1.11192179 -1.11164129 -1.11189651 -1.11194324 -1.1119287  -1.11181188 -1.11202347 -1.11194563 -1.11198425 -1.11199236 -1.11201954 -1.11195874 -1.11209655 -1.11203265 -1.11201501 -1.112046   -1.11211503 -1.11203694 -1.11206317 -1.11209905 -1.11207974 -1.11208487 -1.11209893 -1.11209238 -1.11209142 -1.11211896 -1.11209583 -1.11210334

1.5 1.5 1.5 1.5 1.5 2.  2.5 3.  2.  2.5 2.5 2.  2.  2.5 2.5 2.5 2.  3. 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.

## 3-qubit QD MBE-VQE 

In [ ]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

def _matrix_(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_(coeff):
    return -1j*_matrix_(coeff)@(s_z*dt)

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*ddt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*ddt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10

cz_params = np.array( [1.5472503,  1.4179231,  1.540713,   1.9724044,  1.9253408,  1.3879265,
                     0.8130467,  0.76446086, 1.2703444,  1.8553745,  1.0291328,  1.2492974,
                     0.7880994,  0.3026381,  0.31203356, 0.30834132, 0.9533752,  1.3802187,
                     1.270656,   0.5646567,  0.94619316, 0.97377133, 1.9658349,  0.83277696,
                     1.0190777,  0.90001523, 0.26008993, 0.16526282, 0.22249524, 1.1596956,
                     1.5285202,  0.4919534,  0.01645389, 0.02608137, 0.6504683,  0.31325826,
                     0.4486266,  0.8677286,  1.3571227,  1.4995408,  1.1248059,  0.5996333,
                     0.32797617, 0.54987127])
# params_name = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '0', '1', '2', '3', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119']

def cz_circ(obj_qubit, ctrl_qubit):
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
    # circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
    circ_ += BarrierGate()
    # circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
    # circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
    circ_ += BarrierGate()
    # circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

def arb_circ(prefix='0'):
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate(f'arb0{i}').on(0) for i in range(12)])
    # circ_ += Circuit([gate(0).on(1) for i in range(12)])
    # circ_ += Circuit([gate(0).on(2) for i in range(12)])
    circ_ += BarrierGate()
    # circ_ += Circuit([gate(0).on(0) for i in range(12)])
    circ_ += Circuit([gate(f'arb1{i}').on(1) for i in range(12)])
    # circ_ += Circuit([gate(0).on(2) for i in range(12)])
    circ_ += BarrierGate()
    # circ_ += Circuit([gate(0).on(0) for i in range(12)])
    # circ_ += Circuit([gate(0).on(1) for i in range(12)])
    circ_ += Circuit([gate(f'arb2{i}').on(2) for i in range(12)])
    circ_ += BarrierGate()
    circ_ = add_prefix(circ_, prefix)
    return circ_

def Cz_circ():
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[:10]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([1,0]), gate_c_0(cz_params[21]).on([1,0])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([1,0]), gate_c_1(cz_params[23]).on([1,0])])
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[24:34]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

def qlayer(prefix='0'):
    circ_ = Circuit()
    circ_ += arb_circ('0'+prefix)
    circ_ += cz_circ(1, 0)
    # circ_ += Circuit([gate_0(0).on(2) for i in range(10)])
    # circ_ += arb_circ('1'+prefix)
    circ_ += cz_circ(2, 1)
    # circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    # circ_ += cz_circ(0, 2)
    # circ_ += Circuit([gate_0(0).on(1) for i in range(10)])
    return circ_

class my_simulator: 
    def get_expectation_with_grad(self, hams, circ): # 输入为 hams, circ_right, simulator_left 等，这里就只用 hams 代替。
        sim = Simulator('projectq', 3) 
        
        def grad_ops(inputs): # 输入为各量子门的参数
            h = 1e-4
            grad = []
            sim.reset()
            sim.apply_circuit(circ, inputs)
            exceptation = [sim.get_expectation(ham).real for ham in hams]
            for i in range(inputs.size):
                params_p, params_n = copy.deepcopy(inputs), copy.deepcopy(inputs)
                params_p[i] += h
                params_n[i] -= h
                sim.reset()
                sim.apply_circuit(circ, params_p)
                g_p = [sim.get_expectation(ham) for ham in hams]
                sim.reset()
                sim.apply_circuit(circ, params_n)
                g_n = [sim.get_expectation(ham) for ham in hams]
                grad.append([(p.real - n.real)/(2*h) for p, n in zip(g_p, g_n)])
                sim.reset()
            return exceptation, np.array(grad).T # 运行结果为期望值和梯度
        return grad_ops # MindQuantum 中，这里返回的是一个封装了 hams, grad_ops 等的一个封装器，这里简化为单独的 grad_ops

class ansatz_only_ops(nn.Cell):
    def __init__(self, expectation_with_grad):
        super().__init__()
        self.expectation_with_grad = expectation_with_grad
        self.shape_ops = operations.Shape()
        self.g = None 

    def construct(self, arg):
        fval, g_ans = self.expectation_with_grad(arg.asnumpy())
        self.g = np.real(g_ans)
        return ms.Tensor(np.real(fval), dtype=ms.float32)

    def bprop(self, arg, out, dout): 
        dout = dout.asnumpy()
        grad = dout @ self.g
        return ms.Tensor(grad, dtype=ms.float32)

class ansatz_only_layer(nn.Cell):
    def __init__(self, expectation_with_grad, weight='ones'):
        super().__init__()
        self.evolution = ansatz_only_ops(expectation_with_grad)
        weight_size = len(circ.params_name)
        self.weight = Parameter(initializer(weight, weight_size, dtype=ms.float32), name='ansatz_weight')
        self.abs = ops.Abs()

    def construct(self):
        return self.evolution(self.abs(self.weight))

class MyLoss(LossBase):
    def __init__(self, reduction='mean'):
        super(MyLoss, self).__init__(reduction)
        self.tanh = ops.Tanh()

    def construct(self, logits):
        x = self.tanh(logits)
        out = 0
        for edge in edges:
            out += x[edge[0]] * x[edge[1]]
        return self.get_loss(out)

class MyWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn):
       super(MyWithLossCell, self).__init__(auto_prefix=False)
       self._backbone = backbone
       self._loss_fn = loss_fn

    def construct(self):
       out = self._backbone()
       return self._loss_fn(out)

    @property
    def backbone_network(self):
       return self._backbone

edges = []
node_num = 6
for node in range(node_num-1): 
    edges.append([node, node+1])
edges.append([node_num-1, 0])
for node in range(int(node_num/2)):
    edges.append([node, int(node+node_num/2)])
print(edges)
circ = Circuit()
for i in range(2):
    circ += qlayer(prefix=f'{i}')

hams = [Hamiltonian(QubitOperator('Z0')), Hamiltonian(QubitOperator('X0')), 
        Hamiltonian(QubitOperator('Z1')), Hamiltonian(QubitOperator('X1')),
        Hamiltonian(QubitOperator('Z2')), Hamiltonian(QubitOperator('X2'))]

my_sim = my_simulator()
grad_ops = my_sim.get_expectation_with_grad(hams, circ)

qnet = ansatz_only_layer(grad_ops)
loss = MyLoss()
net_with_criterion = MyWithLossCell(qnet, loss)
opti = Adam(qnet.trainable_params(), learning_rate=0.1) 
net = TrainOneStepCell(net_with_criterion, opti)

round = ops.Round()
loss_list = []
res_list = []
for i in range(100):
    loss = net()
    loss_list.append(loss)
    print(f'\n当前训练次数为：{i}, 损失函数值为：{loss}')
    out = qnet()
    result = 0
    for edge in edges:
        result += (1 - round(out[edge[0]]) * round(out[edge[1]])) / 2
    print('MBE算法计算的最大割数为：', int(result.asnumpy() + 0.5)) #int(res + 0.5))
    res_list.append(int(result.asnumpy() + 0.5)) #int(res + 0.5))

 

In [ ]:
# for i in range(100,201):
#     loss = net()
#     loss_list.append(loss)
#     print(f'\n当前训练次数为：{i}, 损失函数值为：{loss}')
#     out = qnet()
#     result = 0
#     for edge in edges:
#         result += (1 - round(out[edge[0]]) * round(out[edge[1]])) / 2
#     print('MBE算法计算的最大割数为：', int(result.asnumpy() + 0.5)) #int(res + 0.5))
#     res_list.append(int(result.asnumpy() + 0.5)) #int(res + 0.5))
    
loss_array = np.array([])
for i in loss_list:
    loss_array = np.append(loss_array, i.asnumpy()) # 其他方法的收敛值 -3.335446
    
res_array = np.array([])
for i in res_list:
    res_array = np.append(res_array, i)
    
print('loss_array\n', loss_array)
print('\nres_array\n', res_array)

-4.29723059e-07  2.75024533e+00  1.13769400e+00 -6.16364069e-02 -8.08569074e-01 -5.92707038e-01 -7.09778070e-01 -2.18776774e+00 -1.48578441e+00 -1.94576359e+00 -2.08961654e+00 -2.29807091e+00 -2.99075651e+00 -2.54360437e+00 -2.45648241e+00 -3.05445695e+00 -3.18096185e+00 -2.88290739e+00 -2.82773137e+00 -2.95591903e+00 -3.04600668e+00 -3.08430839e+00 -3.21732926e+00 -2.99260306e+00 -3.05293012e+00 -3.10935092e+00 -3.15088177e+00 -3.23413754e+00 -3.21255684e+00 -3.17569113e+00 -3.18600559e+00 -3.21405339e+00 -3.28503275e+00 -3.26810479e+00 -3.24554300e+00 -3.24586272e+00 -3.28752041e+00 -3.29108167e+00 -3.28889012e+00 -3.27797341e+00 -3.30074835e+00 -3.29662442e+00 -3.29751730e+00 -3.31010938e+00 -3.29462767e+00 -3.31268477e+00 -3.31434965e+00 -3.31716490e+00 -3.31385922e+00 -3.32476568e+00 -3.31571841e+00 -3.32363844e+00 -3.32713699e+00 -3.32531166e+00 -3.32381821e+00 -3.32675385e+00 -3.33123255e+00 -3.32730246e+00 -3.32896423e+00 -3.32681131e+00 -3.33236289e+00 -3.33292770e+00 -3.33019233e+00 -3.33160925e+00 -3.33173418e+00 -3.33491611e+00 -3.33343911e+00 -3.33212233e+00 -3.33451581e+00 -3.33464813e+00 -3.33438492e+00 -3.33447790e+00 -3.33415008e+00 -3.33509231e+00 -3.33526206e+00 -3.33465242e+00 -3.33535171e+00 -3.33518028e+00 -3.33552623e+00 -3.33577657e+00 -3.33546686e+00 -3.33564258e+00 -3.33590436e+00 -3.33591604e+00 -3.33592796e+00 -3.33576775e+00 -3.33586550e+00 -3.33627081e+00 -3.33594012e+00 -3.33594370e+00 -3.33616781e+00 -3.33615875e+00 -3.33612537e+00 -3.33619666e+00 -3.33616686e+00 -3.33626604e+00 -3.33625960e+00 -3.33622479e+00 -3.33622622e+00 -3.33631635e+00

0. 4. 5. 5. 5. 5. 8. 6. 5. 7. 6. 9. 6. 8. 9. 9. 8. 8. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.

讨论噪声影响 J

In [1]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

loss_list = []
res_list = []
cz_params = np.array([1.5469222,  1.4163866,  1.5392585,  1.972247,   1.92602,    1.3877034,
                        0.8116179,  0.7628339,  1.2693673,  1.8558168,  1.0307236,  1.2505771,
                        0.7876434,  0.30035374, 0.30896303, 0.30590588, 0.95258,    1.3812971,
                        1.2727345,  0.56576526, 0.94235504, 0.9723476,  1.9667805,  0.8354815,
                        1.0185238,  0.8981452,  0.2593307,  0.16673481, 0.22587535, 1.1638049,
                        1.5322605,  0.4955098,  0.01972879, 0.02794218, 0.64640003, 0.3103799,
                        0.4482256,  0.8699769,  1.3611945,  1.5033041,  1.1262282,  0.59920496,
                        0.32729074, 0.54994607]) # error = 1.3743*10-6
    # params_name = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '0', '1', '2', '3', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119']


    
for dj in [-0.001, -0.0009, -0.0008, -0.0007, -0.0006, -0.0005, -0.0004, -0.0003, -0.0002, -0.0001, 0, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001]:
    def _matrix_(coeff):
        return expm(-1j*((1+dj)*coeff*s_z+s_x)*dt)

    def _diff_matrix_(coeff):
        return -1j*_matrix_((1+dj)*coeff)@(s_z*dt)

    def _matrix_0(coeff):
        return expm(-1j*((1+dj)*coeff*s_z+s_x)*ddt)

    def _diff_matrix_0(coeff):
        return -1j*_matrix_0((1+dj)*coeff)@(s_z*ddt)

    def _matrix_c_0(coeff):
        return expm(-1j*((1+dj)*coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + (1+dj)*coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_0(coeff):
        return -1j*_matrix_c_0((1+dj)*coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

    def _matrix_c_1(coeff):
        return expm(-1j*(kron(s_z, one) + (1+dj)*coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + (1+dj)*coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_1(coeff):
        return -1j*_matrix_c_1((1+dj)*coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

    gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
    gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
    gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
    gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10
    
    def cz_circ(obj_qubit, ctrl_qubit):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
        circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
        circ_ += BarrierGate()
        return circ_

    def arb_circ(prefix='0'):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate(f'arb0{i}').on(0) for i in range(12)])
        circ_ += Circuit([gate(0).on(1) for i in range(12)])
        circ_ += Circuit([gate(0).on(2) for i in range(12)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate(0).on(0) for i in range(12)])
        circ_ += Circuit([gate(f'arb1{i}').on(1) for i in range(12)])
        circ_ += Circuit([gate(0).on(2) for i in range(12)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate(0).on(0) for i in range(12)])
        circ_ += Circuit([gate(0).on(1) for i in range(12)])
        circ_ += Circuit([gate(f'arb2{i}').on(2) for i in range(12)])
        circ_ += BarrierGate()
        circ_ = add_prefix(circ_, prefix)
        return circ_

    def qlayer(prefix='0'):
        circ_ = Circuit()
        circ_ += arb_circ('0'+prefix)
        circ_ += cz_circ(1, 0)
        circ_ += Circuit([gate_0(0).on(2) for i in range(10)])
        circ_ += cz_circ(2, 1)
        circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
        return circ_

    class my_simulator: 
        def get_expectation_with_grad(self, hams, circ): # 输入为 hams, circ_right, simulator_left 等，这里就只用 hams 代替。
            sim = Simulator('projectq', 3) 
            
            def grad_ops(inputs): # 输入为各量子门的参数
                h = 1e-4
                grad = []
                sim.reset()
                sim.apply_circuit(circ, inputs)
                exceptation = [sim.get_expectation(ham).real for ham in hams]
                for i in range(inputs.size):
                    params_p, params_n = copy.deepcopy(inputs), copy.deepcopy(inputs)
                    params_p[i] += h
                    params_n[i] -= h
                    sim.reset()
                    sim.apply_circuit(circ, params_p)
                    g_p = [sim.get_expectation(ham) for ham in hams]
                    sim.reset()
                    sim.apply_circuit(circ, params_n)
                    g_n = [sim.get_expectation(ham) for ham in hams]
                    grad.append([(p.real - n.real)/(2*h) for p, n in zip(g_p, g_n)])
                    sim.reset()
                return exceptation, np.array(grad).T # 运行结果为期望值和梯度
            return grad_ops # MindQuantum 中，这里返回的是一个封装了 hams, grad_ops 等的一个封装器，这里简化为单独的 grad_ops
        
    hams = [Hamiltonian(QubitOperator('Z0')), Hamiltonian(QubitOperator('X0')), 
            Hamiltonian(QubitOperator('Z1')), Hamiltonian(QubitOperator('X1')),
            Hamiltonian(QubitOperator('Z2')), Hamiltonian(QubitOperator('X2'))]

    class ansatz_only_ops(nn.Cell):
        def __init__(self, expectation_with_grad):
            super().__init__()
            self.expectation_with_grad = expectation_with_grad
            self.shape_ops = operations.Shape()
            self.g = None 

        def construct(self, arg):
            fval, g_ans = self.expectation_with_grad(arg.asnumpy())
            self.g = np.real(g_ans)
            return ms.Tensor(np.real(fval), dtype=ms.float32)

        def bprop(self, arg, out, dout): 
            dout = dout.asnumpy()
            grad = dout @ self.g
            return ms.Tensor(grad, dtype=ms.float32)

    class ansatz_only_layer(nn.Cell):
        def __init__(self, expectation_with_grad, weight='ones'):
            super().__init__()
            self.evolution = ansatz_only_ops(expectation_with_grad)
            weight_size = len(circ.params_name)
            self.weight = Parameter(initializer(weight, weight_size, dtype=ms.float32), name='ansatz_weight')
            self.abs = ops.Abs()

        def construct(self):
            return self.evolution(self.abs(self.weight))

    class MyLoss(LossBase):
        def __init__(self, reduction='mean'):
            super(MyLoss, self).__init__(reduction)
            self.tanh = ops.Tanh()

        def construct(self, logits):
            x = self.tanh(logits)
            out = 0
            for edge in edges:
                out += x[edge[0]] * x[edge[1]]
            return self.get_loss(out)

    class MyWithLossCell(nn.Cell):
        def __init__(self, backbone, loss_fn):
            super(MyWithLossCell, self).__init__(auto_prefix=False)
            self._backbone = backbone
            self._loss_fn = loss_fn

        def construct(self):
            out = self._backbone()
            return self._loss_fn(out)

        @property
        def backbone_network(self):
            return self._backbone
        
    edges = []
    node_num = 6
    for node in range(node_num-1): 
        edges.append([node, node+1])
    edges.append([node_num-1, 0])
    for node in range(int(node_num/2)):
        edges.append([node, int(node+node_num/2)])
    circ = Circuit()
    for i in range(2):
        circ += qlayer(prefix=f'{i}')
    my_sim = my_simulator()
    grad_ops = my_sim.get_expectation_with_grad(hams, circ)
    qnet = ansatz_only_layer(grad_ops)
    loss = MyLoss()
    net_with_criterion = MyWithLossCell(qnet, loss)
    opti = Adam(qnet.trainable_params(), learning_rate=0.1) 
    net = TrainOneStepCell(net_with_criterion, opti)

    round = ops.Round()
    for i in range(51):
        loss = net()
    loss_list.append(float(loss.asnumpy()))
    print(f'\n当前dj为：{dj}, 损失函数值为：{loss}')
    out = qnet()
    result = 0
    for edge in edges:
        result += (1 - round(out[edge[0]]) * round(out[edge[1]])) / 2
    print('MBE算法计算的最大割数为：', int(result.asnumpy() + 0.5))
    res_list.append(int(result.asnumpy() + 0.5))
print('loss_list:', loss_list)
print('res_list:', res_list)
 


当前dj为：-0.001, 损失函数值为：-3.2733586
MBE算法计算的最大割数为： 9

当前dj为：-0.0009, 损失函数值为：-3.32582
MBE算法计算的最大割数为： 9

当前dj为：-0.0008, 损失函数值为：-3.3152997
MBE算法计算的最大割数为： 9

当前dj为：-0.0007, 损失函数值为：-3.3224034
MBE算法计算的最大割数为： 9

当前dj为：-0.0006, 损失函数值为：-3.3172963
MBE算法计算的最大割数为： 9

当前dj为：-0.0005, 损失函数值为：-3.1632597
MBE算法计算的最大割数为： 9

当前dj为：-0.0004, 损失函数值为：-3.3035333
MBE算法计算的最大割数为： 9

当前dj为：-0.0003, 损失函数值为：-3.3239388
MBE算法计算的最大割数为： 9

当前dj为：-0.0002, 损失函数值为：-3.304181
MBE算法计算的最大割数为： 9

当前dj为：-0.0001, 损失函数值为：-3.321882
MBE算法计算的最大割数为： 9

当前dj为：0, 损失函数值为：-3.320551
MBE算法计算的最大割数为： 9

当前dj为：0.0001, 损失函数值为：-3.3043017
MBE算法计算的最大割数为： 9

当前dj为：0.0002, 损失函数值为：-3.323684
MBE算法计算的最大割数为： 9

当前dj为：0.0003, 损失函数值为：-3.3159447
MBE算法计算的最大割数为： 9

当前dj为：0.0004, 损失函数值为：-3.3225322
MBE算法计算的最大割数为： 9

当前dj为：0.0005, 损失函数值为：-3.3195026
MBE算法计算的最大割数为： 9

当前dj为：0.0006, 损失函数值为：-3.3179746
MBE算法计算的最大割数为： 9

当前dj为：0.0007, 损失函数值为：-3.2920535
MBE算法计算的最大割数为： 9

当前dj为：0.0008, 损失函数值为：-3.3138962
MBE算法计算的最大割数为： 9

当前dj为：0.0009, 损失函数值为：-3.3210092
MBE算法计算的最大割数为： 9

当

讨论噪声影响 h

In [2]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

loss_list = []
res_list = []
cz_params = np.array([1.5469222,  1.4163866,  1.5392585,  1.972247,   1.92602,    1.3877034,
                        0.8116179,  0.7628339,  1.2693673,  1.8558168,  1.0307236,  1.2505771,
                        0.7876434,  0.30035374, 0.30896303, 0.30590588, 0.95258,    1.3812971,
                        1.2727345,  0.56576526, 0.94235504, 0.9723476,  1.9667805,  0.8354815,
                        1.0185238,  0.8981452,  0.2593307,  0.16673481, 0.22587535, 1.1638049,
                        1.5322605,  0.4955098,  0.01972879, 0.02794218, 0.64640003, 0.3103799,
                        0.4482256,  0.8699769,  1.3611945,  1.5033041,  1.1262282,  0.59920496,
                        0.32729074, 0.54994607]) # error = 1.3743*10-6
    # params_name = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '0', '1', '2', '3', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119']


    
for dh in[0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001]: # -0.001, -0.0009, -0.0008, -0.0007, -0.0006, -0.0005, -0.0004, -0.0003, -0.0002, -0.0001, 0, 0.0001, 
    def _matrix_(coeff):
        return expm(-1j*(coeff*s_z+(1+dh)*s_x)*dt)

    def _diff_matrix_(coeff):
        return -1j*_matrix_(coeff)@(s_z*dt)

    def _matrix_0(coeff):
        return expm(-1j*(coeff*s_z+(1+dh)*s_x)*ddt)

    def _diff_matrix_0(coeff):
        return -1j*_matrix_0(coeff)@(s_z*ddt)

    def _matrix_c_0(coeff):
        return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron((1+dh)*s_x, one) + kron(one, (1+dh)*s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_0(coeff):
        return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

    def _matrix_c_1(coeff):
        return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron((1+dh)*s_x, one) + kron(one, (1+dh)*s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_1(coeff):
        return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

    gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
    gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
    gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
    gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10
    
    def cz_circ(obj_qubit, ctrl_qubit):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
        circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
        circ_ += BarrierGate()
        return circ_

    def arb_circ(prefix='0'):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate(f'arb0{i}').on(0) for i in range(12)])
        circ_ += Circuit([gate(0).on(1) for i in range(12)])
        circ_ += Circuit([gate(0).on(2) for i in range(12)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate(0).on(0) for i in range(12)])
        circ_ += Circuit([gate(f'arb1{i}').on(1) for i in range(12)])
        circ_ += Circuit([gate(0).on(2) for i in range(12)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate(0).on(0) for i in range(12)])
        circ_ += Circuit([gate(0).on(1) for i in range(12)])
        circ_ += Circuit([gate(f'arb2{i}').on(2) for i in range(12)])
        circ_ += BarrierGate()
        circ_ = add_prefix(circ_, prefix)
        return circ_

    def qlayer(prefix='0'):
        circ_ = Circuit()
        circ_ += arb_circ('0'+prefix)
        circ_ += cz_circ(1, 0)
        circ_ += Circuit([gate_0(0).on(2) for i in range(10)])
        circ_ += cz_circ(2, 1)
        circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
        return circ_

    class my_simulator: 
        def get_expectation_with_grad(self, hams, circ): # 输入为 hams, circ_right, simulator_left 等，这里就只用 hams 代替。
            sim = Simulator('projectq', 3) 
            
            def grad_ops(inputs): # 输入为各量子门的参数
                h = 1e-4
                grad = []
                sim.reset()
                sim.apply_circuit(circ, inputs)
                exceptation = [sim.get_expectation(ham).real for ham in hams]
                for i in range(inputs.size):
                    params_p, params_n = copy.deepcopy(inputs), copy.deepcopy(inputs)
                    params_p[i] += h
                    params_n[i] -= h
                    sim.reset()
                    sim.apply_circuit(circ, params_p)
                    g_p = [sim.get_expectation(ham) for ham in hams]
                    sim.reset()
                    sim.apply_circuit(circ, params_n)
                    g_n = [sim.get_expectation(ham) for ham in hams]
                    grad.append([(p.real - n.real)/(2*h) for p, n in zip(g_p, g_n)])
                    sim.reset()
                return exceptation, np.array(grad).T # 运行结果为期望值和梯度
            return grad_ops # MindQuantum 中，这里返回的是一个封装了 hams, grad_ops 等的一个封装器，这里简化为单独的 grad_ops
        
    hams = [Hamiltonian(QubitOperator('Z0')), Hamiltonian(QubitOperator('X0')), 
            Hamiltonian(QubitOperator('Z1')), Hamiltonian(QubitOperator('X1')),
            Hamiltonian(QubitOperator('Z2')), Hamiltonian(QubitOperator('X2'))]

    class ansatz_only_ops(nn.Cell):
        def __init__(self, expectation_with_grad):
            super().__init__()
            self.expectation_with_grad = expectation_with_grad
            self.shape_ops = operations.Shape()
            self.g = None 

        def construct(self, arg):
            fval, g_ans = self.expectation_with_grad(arg.asnumpy())
            self.g = np.real(g_ans)
            return ms.Tensor(np.real(fval), dtype=ms.float32)

        def bprop(self, arg, out, dout): 
            dout = dout.asnumpy()
            grad = dout @ self.g
            return ms.Tensor(grad, dtype=ms.float32)

    class ansatz_only_layer(nn.Cell):
        def __init__(self, expectation_with_grad, weight='ones'):
            super().__init__()
            self.evolution = ansatz_only_ops(expectation_with_grad)
            weight_size = len(circ.params_name)
            self.weight = Parameter(initializer(weight, weight_size, dtype=ms.float32), name='ansatz_weight')
            self.abs = ops.Abs()

        def construct(self):
            return self.evolution(self.abs(self.weight))

    class MyLoss(LossBase):
        def __init__(self, reduction='mean'):
            super(MyLoss, self).__init__(reduction)
            self.tanh = ops.Tanh()

        def construct(self, logits):
            x = self.tanh(logits)
            out = 0
            for edge in edges:
                out += x[edge[0]] * x[edge[1]]
            return self.get_loss(out)

    class MyWithLossCell(nn.Cell):
        def __init__(self, backbone, loss_fn):
            super(MyWithLossCell, self).__init__(auto_prefix=False)
            self._backbone = backbone
            self._loss_fn = loss_fn

        def construct(self):
            out = self._backbone()
            return self._loss_fn(out)

        @property
        def backbone_network(self):
            return self._backbone
        
    edges = []
    node_num = 6
    for node in range(node_num-1): 
        edges.append([node, node+1])
    edges.append([node_num-1, 0])
    for node in range(int(node_num/2)):
        edges.append([node, int(node+node_num/2)])
    circ = Circuit()
    for i in range(2):
        circ += qlayer(prefix=f'{i}')
    my_sim = my_simulator()
    grad_ops = my_sim.get_expectation_with_grad(hams, circ)
    qnet = ansatz_only_layer(grad_ops)
    loss = MyLoss()
    net_with_criterion = MyWithLossCell(qnet, loss)
    opti = Adam(qnet.trainable_params(), learning_rate=0.1) 
    net = TrainOneStepCell(net_with_criterion, opti)

    round = ops.Round()
    for i in range(51):
        loss = net()
    loss_list.append(float(loss.asnumpy()))
    print(f'\n当前dh为：{dh}, 损失函数值为：{loss}')
    out = qnet()
    result = 0
    for edge in edges:
        result += (1 - round(out[edge[0]]) * round(out[edge[1]])) / 2
    print('MBE算法计算的最大割数为：', int(result.asnumpy() + 0.5))
    res_list.append(int(result.asnumpy() + 0.5))
print('loss_list:', loss_list)
print('res_list:', res_list)
# -3.2765427 -3.0640457 -3.3120396 -3.309067 -3.314459 -3.3166137 -3.3151793 -3.2924461 -3.2425184 -3.3103433 -3.320551 -3.3103735
 


当前dh为：0.0002, 损失函数值为：-3.324049
MBE算法计算的最大割数为： 9

当前dh为：0.0003, 损失函数值为：-3.3151553
MBE算法计算的最大割数为： 9

当前dh为：0.0004, 损失函数值为：-3.3133185
MBE算法计算的最大割数为： 9

当前dh为：0.0005, 损失函数值为：-3.3088465
MBE算法计算的最大割数为： 9

当前dh为：0.0006, 损失函数值为：-3.3120005
MBE算法计算的最大割数为： 9

当前dh为：0.0007, 损失函数值为：-3.3148396
MBE算法计算的最大割数为： 9

当前dh为：0.0008, 损失函数值为：-3.3226714
MBE算法计算的最大割数为： 9

当前dh为：0.0009, 损失函数值为：-3.3040495
MBE算法计算的最大割数为： 9

当前dh为：0.001, 损失函数值为：-3.3175426
MBE算法计算的最大割数为： 9
loss_list: [-3.3240489959716797, -3.315155267715454, -3.3133184909820557, -3.3088464736938477, -3.3120005130767822, -3.3148396015167236, -3.322671413421631, -3.304049491882324, -3.317542552947998]
res_list: [9, 9, 9, 9, 9, 9, 9, 9, 9]


In [ ]:

当前dh为：-0.001, 损失函数值为：-3.2765427
MBE算法计算的最大割数为： 9

当前dh为：-0.0009, 损失函数值为：-3.0640457
MBE算法计算的最大割数为： 9

当前dh为：-0.0008, 损失函数值为：-3.3120396
MBE算法计算的最大割数为： 9

当前dh为：-0.0007, 损失函数值为：-3.309067
MBE算法计算的最大割数为： 9

当前dh为：-0.0006, 损失函数值为：-3.314459
MBE算法计算的最大割数为： 9

当前dh为：-0.0005, 损失函数值为：-3.3166137
MBE算法计算的最大割数为： 9

当前dh为：-0.0004, 损失函数值为：-3.3151793
MBE算法计算的最大割数为： 9

当前dh为：-0.0003, 损失函数值为：-3.2924461
MBE算法计算的最大割数为： 9

当前dh为：-0.0002, 损失函数值为：-3.2425184
MBE算法计算的最大割数为： 9

当前dh为：-0.0001, 损失函数值为：-3.3103433
MBE算法计算的最大割数为： 9

当前dh为：0, 损失函数值为：-3.320551
MBE算法计算的最大割数为： 9

当前dh为：0.0001, 损失函数值为：-3.3103735
MBE算法计算的最大割数为： 9
